#### glu層を入れたモデル
image_featの標準化、チャネル正規化, subject_id = 0, im_feat標準化なしin loss  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),  
hid_dim=128, p_drop=0.1, lr=0.001   2975(101)  
hid_dim=128, p_drop=0.1, lr=0.0005  3370(102)  
hid_dim=128, p_drop=0.1, lr=0.0002  3180(103)  
hid_dim=128, p_drop=0.1, lr=0.0001  2391(104)  

hid_dim=64, p_drop=0.1, lr=0.001   3235(105)  
hid_dim=64, p_drop=0.1, lr=0.0005  3307(106)  
hid_dim=64, p_drop=0.1, lr=0.0002  2975(107)  
hid_dim=64, p_drop=0.1, lr=0.0001  2857(108)  

hid_dim=32, p_drop=0.1, lr=0.001   3235(109)  
hid_dim=32, p_drop=0.1, lr=0.0005  3259(110)  
hid_dim=32, p_drop=0.1, lr=0.0002  2999(111)  
hid_dim=32, p_drop=0.1, lr=0.0001  2833(112)  
あまり上がらない...

Dilated conv する


batch_sizeを大きくする  

image_featの標準化、チャネル正規化, subject_id = 0, im_feat標準化なしin loss  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=128   3330(113)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=256   3546(114)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512   3827(115)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=1024   5189(116)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=2048   6239(平均的には3500ぐらい)(117) 未学習  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=4096   4447(118)未学習  

batch_sizeを大きくすると、最大のスコアは上がるが、揺れが激しくなる  
batch_size=2048では、train_accが0.02レベル(val_accより小さい)だけどなぜ？  
→ batch_sizeを大きくすると、スコアが上がる理由は、batch内で損失関数を計算していて、不正解を多く知れるからでは？  
→ 逆に、正解を不正解にする力も間違えて入ってしまう。これが学習を妨げているのでは？  


学習率を小さくすれば、振動が和らぐ？



データを増やす（）  
自己強化学習(MAE)  どのようにmaskをかけるべきか?  


logit_scaleをパラメータ化した
image_featの標準化、チャネル正規化, subject_id = 0, im_feat標準化なしin loss  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=128   3217(119)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=256   3462(120)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512   4774(121)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=1024   5365(122)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=2048   6548(123) 未学習  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=4096   4472(124)未学習  

全データを用いる  
image_featの標準化、チャネル正規化, im_feat標準化なしin loss, logit_scaleをパラメータ化  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=128   0.034()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=256   0.033()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512   0.032()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=1024   0.032()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=2048   0.030()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=4096   ()  

convをsubject_idx毎に学習し、最後のDensLayerは共通化  
image_featの標準化、チャネル正規化, im_feat標準化なしin loss, logit_scaleをパラメータ化  
ConvBlock(in_channels, hid_dim), ConvBlock(hid_dim, hid_dim),glu層入れてない  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=128   0.046 ()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=256   0.047 ()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=512   0.048 ()  public:0.04771(修正後)  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=1024   0.044()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=2048   0.044()  
hid_dim=128, p_drop=0.1, lr=0.001, batch_size=4096   0.045()  public:0.00487(おかしい、修正前)  
10回以上改善しなかったら打ち切り  